In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
import os

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# define train set
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('./drive/My Drive/DACON/data_file/train.csv')
test = pd.read_csv('./drive/My Drive/DACON/data_file/test.csv')
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
rot_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=45, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     # 블로그 펌 ㅎ
    zoom_range=0,      # 마찬가지 블로그 펌 ㅎ
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

trans_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.2, # 블로그 펌
    height_shift_range=0.2,# 블로그 펌
    brightness_range=None,
    shear_range=0,     # 블로그 펌
    zoom_range=0,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

shear_zoom_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.2,     # 블로그 펌
    zoom_range=0.2,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

flip_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     # 블로그 펌
    zoom_range=0,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=True, # 뒤집기
    vertical_flip=True,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)
df = train_copy
new_data_set = []
num_of_training_set = df.shape[0]

for i in range(num_of_training_set//4):
    rand_1 = np.random.randint(num_of_training_set)
    rand_2 = np.random.randint(num_of_training_set)
    rand_3 = np.random.randint(num_of_training_set)
    rand_4 = np.random.randint(num_of_training_set)
   
    for j in range(5):
        # rotation
        _rot = rot_gen.flow( np.array(df.iloc[rand_1,3:]).reshape(1,28,28,1) ).next().reshape(784,)
        new_data_set += [[
            df.iloc[rand_1,1],
            df.iloc[rand_1,2],
        ] + list(_rot)]
        # translation
        _trans = trans_gen.flow( np.array(df.iloc[rand_2,3:]).reshape(1,28,28,1) ).next().reshape(784,)
        new_data_set += [[
            df.iloc[rand_2,1],
            df.iloc[rand_2,2],
        ] + list(_trans)]
        # shear / zoom
        _shear = shear_zoom_gen.flow( np.array(df.iloc[rand_3,3:]).reshape(1,28,28,1) ).next().reshape(784,)
        new_data_set += [[
            df.iloc[rand_3,1],
            df.iloc[rand_3,2],
        ] + list(_shear)]
        # flip
        _flip = flip_gen.flow( np.array(df.iloc[rand_4,3:]).reshape(1,28,28,1) ).next().reshape(784,)
        new_data_set += [[
            df.iloc[rand_4,1],
            df.iloc[rand_4,2],
        ] + list(_flip)]

In [ ]:
columns = ['digit', 'letter'] + [str(x) for x in range(784)]
aug = pd.DataFrame(new_data_set, columns=columns)

In [ ]:
aug.head()

In [ ]:
train_norm = pd.concat([ train_copy.iloc[:,1:3], np.divide(train_copy.iloc[:,3:],255) ],axis=1)


train_aug = pd.concat([train_norm,aug])
x_train = train_aug.iloc[:,2:].values.reshape(-1,28,28,1)

y_train = train_aug['digit'].copy()
y_train = to_categorical(y_train,num_classes = 10)

# split training and validation set
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=15)


In [ ]:
def get_model():
    N = 64
    model = Sequential()

    model.add(Conv2D(filters = N, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu', input_shape = (28,28,1)))
    model.add(Conv2D(filters = N, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu'))
                
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 2*N, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model.add(Conv2D(filters = 2*N, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(4*N, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = "softmax"))

    return model

In [ ]:
epochs = 50 
batch_size = 50
num = 5
model_list = []

for i in range(num):

    model = get_model()

    MODEL_SAVE_FOLDER_PATH = './drive/My Drive/DACON/model_ensemble/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + '{val_loss: .4f}-{val_accuracy:.4f}.hdf5'

    # callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
    mcp_save = ModelCheckpoint(filepath = model_path, save_best_only=True, monitor='val_loss', mode='min', verbose=1)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='min')

    optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

    model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

    model.fit(x_train, y_train, batch_size=batch_size, epochs = epochs, 
                validation_data = (x_val,y_val),
                steps_per_epoch=x_train.shape[0]// batch_size, 
                callbacks=[early_stopping,mcp_save,reduce_lr_loss])
    model_list.append(model)



In [ ]:
def ensemble(input_imgs):
    pred = []
    L = input_imgs.shape[0]
    label_list = np.zeros((L,10))
    for i in range(num):
        label = model_list[i].predict_on_batch( np.array(input_imgs).reshape(-1,28,28,1).astype(np.float32) )
        label_list += label
    for j in range(len(label_list)):
        pred.append( np.argmax(label_list[j]) )

    return pred

In [ ]:
x = np.divide(test_copy.iloc[:5,2:].values,255)
x = x.reshape(-1,28,28,1)
a = ensemble(x)
print(a)

In [ ]:
x_test = np.divide(test_copy.iloc[:,2:].values,255)
x_test = x_test.reshape(-1,28,28,1)
pred = ensemble(x_test)
data = {'id':test_copy['id'], 'digit':pred}
submission = DataFrame(data)
submission.to_csv('./drive/My Drive/DACON/submission/submission_cnn.csv', index=False)